 В нашем распоряжении имеется дата-сет "Книга отзывов пансионата "Родник Алтая" за определенный промежуток времени. Отзывы размечены как: положительный, отрицательный, нейтральный и предложение. Наша задача по имеющимся историческим данным создать и обучить модель, которая сможет предсказывать оценку тональности отзыва.


### План работ:
1. Подготовка данных
2. Очистка данных
3. Проведение токенизации текста с помощью предобученной модели RUbert2&
4. Обучение 3 моделей.
5. Выбор модели для выпуска в продакшн по показателям f1, accuracy, ROC_auc.


# 1. Подготовка данных
Сначала проведем установку всех необхоимых библиотек

In [117]:
!pip install BertTokenizer
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement BertTokenizer (from versions: none)
ERROR: No matching distribution found for BertTokenizer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
# Загружаем необходимые библиотеки

import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook

import re

from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report

import lightgbm
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
# Загружаем и смотрим на наши данные

data = pd.read_excel('/content/Rodniky_otzivi.xls', index_col=1)
data.info()
data.head(2)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4989 entries, 2020-01-13 to 2020-12-28
Data columns (total 10 columns):
 #   Column                                                     Non-Null Count  Dtype         
---  ------                                                     --------------  -----         
 0   Источник                                                   4989 non-null   object        
 1   N                                                          4989 non-null   int64         
 2   Дата отзыва                                                4904 non-null   object        
 3   Первичный текст в отзыве                                   4989 non-null   object        
 4   Подразделение                                              4989 non-null   object        
 5   Тональность                                                4989 non-null   object        
 6   Тег                                                        4920 non-null   object        
 7   Комментарий: ес

,Источник,N,Дата отзыва,Первичный текст в отзыве,Подразделение,Тональность,Тег,"Комментарий: если есть (ссылка, ответ, отработка и т.д.)",Рег№,Месяц_отчета
Дата реестра,,,,,,,,,,
2020-01-13,"Отзывы по результатам обходов, встреч, книг жалоб",2,2020-01-01 00:00:00,03.01.2020. Обратилась с замечаниями.1) В басс...,Внутренний_бассейн,Отрицательно,"Много народа, тесно",NaN,160698,2020-01-01
2020-01-13,"Отзывы по результатам обходов, встреч, книг жалоб",2,2020-01-01 00:00:00,03.01.2020. Обратилась с замечаниями.1) В басс...,Служба_питания,Отрицательно,Качество блюд и напитков,NaN,160698,2020-01-01


Датасет более менее сбалансирован - имеются пропуски в некоторых колонках, некоторые из которых нам не важны для дальнейшей работы. Однако, удалять данные строки не будем, т.к. при их удалении мы удалим даные в наших целевых колонках. Целевые колонки - "Первичный текст в отзыве" и "Тональность" (в каждом 4989 строк).
Проведем заполнение пропусков в тех колонках, где это необходимо.


In [122]:
# Заполняем пропуски
data['Дата отзыва'] = data['Дата отзыва'].fillna(value='Без даты')
data['Тег'] = data['Тег'].fillna(value='Без тега')
data['Комментарий: если есть  (ссылка, ответ, отработка и т.д.)'] = data['Комментарий: если есть  (ссылка, ответ, отработка и т.д.)'].fillna(value='Без комментариев')
data['Рег№'] = data['Рег№'].fillna(value='Без №')                                                                                                                                            

In [123]:
# Проверка
data.isna().sum()

Источник                                                     0
N                                                            0
Дата отзыва                                                  0
Первичный текст в отзыве                                     0
Подразделение                                                0
Тональность                                                  0
Тег                                                          0
Комментарий: если есть  (ссылка, ответ, отработка и т.д.)    0
Рег№                                                         0
Месяц_отчета                                                 0
dtype: int64

In [124]:
# Смотрим на наши  Lables
data['Тональность'].unique

<bound method Series.unique of Дата реестра
2020-01-13    Отрицательно
2020-01-13    Отрицательно
2020-01-13    Отрицательно
2020-01-13    Отрицательно
2020-01-13    Отрицательно
                  ...     
2020-12-28    Положительно
2020-12-28    Положительно
2020-12-28      Нейтрально
2020-12-28    Положительно
2020-12-28     Предложение
Name: Тональность, Length: 4989, dtype: object>

Пропуски в данных заменили, в колонке текст у нас текст с лишними символами, которые необходимо почистить. 
После создадим эмбединги и векторизуем текст с помощью предобученной модели РуБерт_2, данная модель является более легковесной, чем BERT(ее уменьшенная версия), обучена на   корпусе Яндекс Переводчика , OPUS-100 и Tatoeba , с использованием MLM loss (дистиллированного из bert-base-multilingual-cased ), потери при ранжировании переводов и [CLS]эмбеддингов, перегнанных из LaBSE , rubert-base-cased-sentence , Laser.
1. Переведем колонку "Тональность" в цифровой вид:

In [125]:
print('Проведем разделение тональности на категории')
def opred_ton_digit(row):
    ##пишем функцию, которая возвращает значение согласно следующему правилу:
    ###Отрицательный отзыв - 0
    ###Положительный отзыв - 1
    ###Предложениеб Нейтрально - 2
    ton = row['Тональность']
    
    if ton == 'Отрицательно': 
            return '0'
        
    if ton == 'Положительно': 
            return '1'
        
    if ton == 'Предложение': 

            return '2'
    if ton == 'Нейтрально': 
            return '2'

row_values = [0] 
row_columns = ['Тональность'] 

row = pd.Series(data=row_values, index=row_columns)
print(opred_ton_digit(row)) # проверка работы функции
    
data['Тональность_цифра'] = data.apply(opred_ton_digit, axis=1)  
data_1 = data['Тональность_цифра'].value_counts()
print(data_1)

Проведем разделение тональности на категории
None
1    2753
2    1471
0     762
Name: Тональность_цифра, dtype: int64


In [126]:
data = data.dropna()

In [127]:
data.head(2)

,Источник,N,Дата отзыва,Первичный текст в отзыве,Подразделение,Тональность,Тег,"Комментарий: если есть (ссылка, ответ, отработка и т.д.)",Рег№,Месяц_отчета,Тональность_цифра
Дата реестра,,,,,,,,,,,
2020-01-13,"Отзывы по результатам обходов, встреч, книг жалоб",2,2020-01-01 00:00:00,03.01.2020. Обратилась с замечаниями.1) В басс...,Внутренний_бассейн,Отрицательно,"Много народа, тесно",Без комментариев,160698,2020-01-01,0
2020-01-13,"Отзывы по результатам обходов, встреч, книг жалоб",2,2020-01-01 00:00:00,03.01.2020. Обратилась с замечаниями.1) В басс...,Служба_питания,Отрицательно,Качество блюд и напитков,Без комментариев,160698,2020-01-01,0


In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4986 entries, 2020-01-13 to 2020-12-28
Data columns (total 11 columns):
 #   Column                                                     Non-Null Count  Dtype         
---  ------                                                     --------------  -----         
 0   Источник                                                   4986 non-null   object        
 1   N                                                          4986 non-null   int64         
 2   Дата отзыва                                                4986 non-null   object        
 3   Первичный текст в отзыве                                   4986 non-null   object        
 4   Подразделение                                              4986 non-null   object        
 5   Тональность                                                4986 non-null   object        
 6   Тег                                                        4986 non-null   object        
 7   Комментарий: ес

In [129]:
 # Столбец имеет тип Object - меняем на числовой
data['Тональность_цифра'] = pd.to_numeric(data['Тональность_цифра'], errors='coerce')

In [130]:
# Смотрим наш текс
#text = data['Первичный текст в отзыве'].values.astype('U')
#text

Чистим текст от ненужных символов

In [131]:
# Функция очистки
def clear_text(text):
    t2 = text.split() # создаем дополнительные пробелы
    return " ".join(t2) # убираем лишние пробелы

In [132]:
# Запускаем цикл очистки: задаем символы, которые должны остаться, далее вызываем функцию очистки , 
# которая в места, где были ненужные символы добавит пробелы и потом с помощью joint уберет лишние пробелы
for i in range(len(data)):
    data['Первичный текст в отзыве'][i] = re.sub(r'[^а-яA-я]', ' ', data['Первичный текст в отзыве'][i])
    data['Первичный текст в отзыве'][i] = clear_text(data['Первичный текст в отзыве'][i])

# Проверка очистки

print(data['Первичный текст в отзыве'][0])
print('=============')
print(data['Первичный текст в отзыве'][1])

Обратилась с замечаниями В бассейне много отдыхающих в воде волосы Малые блюда и рыба не свежая В номере грязные чанки перемыли после замечания Банкет хуже прошлых лет
Обратилась с замечаниями В бассейне много отдыхающих в воде волосы Малые блюда и рыба не свежая В номере грязные чанки перемыли после замечания Банкет хуже прошлых лет


In [133]:
# Подровняем нашу таблицу до того как начнем создавать эмбединги
batch_1 = data[:4900]


In [134]:
batch_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4900 entries, 2020-01-13 to 2020-12-21
Data columns (total 11 columns):
 #   Column                                                     Non-Null Count  Dtype         
---  ------                                                     --------------  -----         
 0   Источник                                                   4900 non-null   object        
 1   N                                                          4900 non-null   int64         
 2   Дата отзыва                                                4900 non-null   object        
 3   Первичный текст в отзыве                                   4900 non-null   object        
 4   Подразделение                                              4900 non-null   object        
 5   Тональность                                                4900 non-null   object        
 6   Тег                                                        4900 non-null   object        
 7   Комментарий: ес

Смотрим на распределение классов

In [135]:
batch_1['Тональность_цифра'].value_counts()

1    2701
2    1449
0     750
Name: Тональность_цифра, dtype: int64

Классы несбалансированы - при разделении на выборки необходимо использовать стратификацию.

In [136]:
batch_1['Тональность_цифра'].value_counts().sum()

4900

In [137]:
batch_1 = batch_1[['Первичный текст в отзыве','Тональность_цифра']].reset_index(drop=True) 

In [138]:

print(batch_1)


                               Первичный текст в отзыве  Тональность_цифра
0     Обратилась с замечаниями В бассейне много отды...                  0
1     Обратилась с замечаниями В бассейне много отды...                  0
2     Обратилась с замечаниями В бассейне много отды...                  0
3     Обратилась с замечаниями В бассейне много отды...                  0
4                     По приезду в не предоставили обед                  0
...                                                 ...                ...
4895                              алена и павел молодцы                  1
4896       павел пайзеров молодец Убрать ограничение до                  1
4897  Огромное спасибо Евгению за суперские трениров...                  1
4898            Спасибо за спорт йога акваэробика зумба                  1
4899  Спасибо Павлу который ведет мафию очень компет...                  1

[4900 rows x 2 columns]


#### Создание эмбедингов с помощью RUBERT_2


In [139]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [140]:
tokenized = batch_1['Первичный текст в отзыве'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=312)))

In [141]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [142]:
np.array(padded).shape

(4900, 312)

In [143]:

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('Размер', model, tokenizer).shape)


(312,)


In [144]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(4900, 312)

In [145]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask_batch)        
        
        embeddings.append((last_hidden_states[0][:,0,:]).numpy())

  0%|          | 0/49 [00:00<?, ?it/s]

 ### Создаем FEATURES LABLES. Разделяем выборку на тренировочную и тестовую.
 Для разделения используем стратификацию в силу несблансированности классов.

In [146]:
features = np.concatenate(embeddings)

In [147]:
labels = data['Тональность_цифра'].head(4900)
len(labels)

4900

In [148]:
len(features)

4900

In [149]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state=42, stratify=labels)

### Обучение моделей: LogisticRegression, DecisionTreeClassifier, LGBM

In [150]:

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [151]:
lr_clf.score(test_features, test_labels)

0.8255102040816327

In [152]:
predict = lr_clf.predict(test_features)

f1_score(test_labels, predict, average='macro')

0.7530844669408415

In [153]:
y_pred = lr_clf.predict(test_features)
y_pred
print("LogisticRegression", classification_report(test_labels, y_pred, labels=[0, 1, 2]))

LogisticRegression               precision    recall  f1-score   support

           0       0.59      0.55      0.57       150
           1       0.92      0.93      0.92       540
           2       0.77      0.77      0.77       290

    accuracy                           0.83       980
   macro avg       0.76      0.75      0.75       980
weighted avg       0.82      0.83      0.82       980



In [ ]:
y_pred

In [155]:
roc_auc_score(test_labels, lr_clf.predict_proba(test_features), multi_class='ovr')

0.9406064327983671

### DecisionTreeClassifier


In [157]:
# Дерево решений с подбором гиперпараметров
best_model = None
best_result = 0
for depth in range(1, 6):
    clf = DecisionTreeClassifier(random_state=12345, max_depth = depth) 
    clf.fit(train_features, train_labels) # обучите модель
    predictions_DT = clf.predict(test_features) 
    
    result = accuracy_score(test_labels, predictions_DT) 
    if result > best_result:
        best_model = model
        best_result = result
        
print('Дерево решений: значение Меры accuracy_score:', best_result, 'Максимальная глубина', depth)

Дерево решений: значение Меры accuracy_score: 0.7448979591836735 Максимальная глубина 5


In [160]:
print("DecisionTreeClassifier", classification_report(test_labels, predictions_DT, labels=[0, 1, 2]))

DecisionTreeClassifier               precision    recall  f1-score   support

           0       0.42      0.31      0.35       150
           1       0.85      0.90      0.87       540
           2       0.67      0.68      0.68       290

    accuracy                           0.74       980
   macro avg       0.64      0.63      0.63       980
weighted avg       0.73      0.74      0.73       980



In [162]:
roc_auc_score(test_labels, clf.predict_proba(test_features), multi_class='ovr')

0.8508398867277651

 ## lightgbm

In [164]:

model_LGBM = lightgbm.LGBMClassifier(learning_rate = 0.1, num_leaves = 100)

In [165]:
model_LGBM.fit(train_features, train_labels)

LGBMClassifier(num_leaves=100)

In [166]:

predict_LGBM = model_LGBM.predict(test_features)


In [167]:
accuracy_score(test_labels, predict_LGBM)
print('Значение accuracy_score модель LGBM', accuracy_score(test_labels, predict_LGBM))

Значение accuracy_score модель LGBM 0.8367346938775511


In [ ]:
predict_LGBM

In [170]:
print('LGBM', classification_report(test_labels, predict_LGBM, labels=[0, 1, 2]))

LGBM               precision    recall  f1-score   support

           0       0.66      0.55      0.60       150
           1       0.91      0.94      0.92       540
           2       0.78      0.79      0.79       290

    accuracy                           0.84       980
   macro avg       0.78      0.76      0.77       980
weighted avg       0.83      0.84      0.83       980



In [171]:
roc_auc_score(test_labels, model_LGBM.predict_proba(test_features), multi_class='ovr')

0.9367945800993057

### Вывод

По результатам проведенных исследований 1 место занимает модель lightgbm c accuracy 0.84, f1 0.77 roc_auc - 0.937.
Далее LogisticRegression 0.83, мера f1 - 0.75 roc_auc - 0.941.
3 место DesicionTreeClassify 0.74, f1 0.63, roc_auc - . 0.851